In [ ]:
import findspark
from pyspark.sql import SparkSession

findspark.init()


def init_spark():
    spark = SparkSession.builder.appName("HelloWorld").getOrCreate()
    sc = spark.sparkContext
    return spark, sc


def main():
    spark, sc = init_spark()
    # nums = sc.parallelize([1, 2, 3, 4])
    # print(nums.map(lambda x: x * x).collect())
    dist_file = sc.textFile('Libraries.io-open-data-1.4.0.tar.gz', 5000)
    size = dist_file.map(lambda s: len(s)).reduce(lambda a, b: a+b)
    print(size)


if __name__ == '__main__':
    main()

